In [10]:
!pip install gradio  
!pip install  transformers
!pip install nltk



In [14]:

import gradio as gr
import re
import warnings

#from gradio.mix import Parallel
from nltk.tokenize import sent_tokenize
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    Wav2Vec2ForCTC,
    Wav2Vec2Tokenizer,
    pipeline,
)

warnings.filterwarnings('ignore')

**Load Data and Define Summarizing functions**

In [15]:
import pickle

with open('cleaned_data.pkl', 'rb') as file:
     dataset = pickle.load(file)

**a. Summarization via Hugging Face Pipeline**


Hugging Face's pipeline allows you to load up several summarization models, from FB's Bart to Google's T5. 

In [30]:
import sentencepiece as spm
# Load the slow tokenizer
slow_tokenizer = YourSlowTokenizer()

# Convert to fast tokenizer using SentencePiece
sp_tokenizer = spm.SentencePieceProcessor()
sp_tokenizer.Load("path/to/sentencepiece_model.model")

# Now you can use sp_tokenizer for tokenization
tokens = sp_tokenizer.EncodeAsPieces("Your input text")


pipeline_summ = pipeline(
    "summarization",
    model="facebook/bart-large-cnn", # one can also use"t5-small" etc 
    tokenizer="facebook/bart-large-cnn",
    framework="pt",
)

# First of 2 summarization function
def fb_summarizer(dataset):
    #input_text = dataset
    results = pipeline_summ(dataset)
    return [result["summary_text"] for result in results]

# First of 2 Gradio apps to use in "parallel"
summary1 = gr.Interface(
    fn=fb_summarizer,
    inputs=gr.Textbox(),
    outputs=gr.Textbox(label="Summary by FB/Bart-large"),
)

**b. Summarization using Hugging  Face's Model Hub**

In [33]:
model_name = "google/pegasus-cnn_dailymail" 

# Second of 2 summarization function
def google_summarizer(dataset):
    input_texts = dataset
    
    tokenizer_pegasus = AutoTokenizer.from_pretrained(model_name)

    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    batch = tokenizer_pegasus.prepare_seq2seq_batch(
        input_texts, truncation=True, padding="longest", return_tensors="pt"
    )
    translated = model_pegasus.generate(**batch)

    pegasus_summary = tokenizer_pegasus.batch_decode(
        translated, skip_special_tokens=True
    )

    return pegasus_summary[0]

# Second of 2 Gradio apps to use in "parallel"
summary2 = gr.Interface(
    fn=google_summarizer,
    inputs=gr.Textbox(),
    outputs=gr.Textbox(label="Summary by Google/Pegasus-CNN-Dailymail"),
)

**2. Launch the Models in Parallel**

In [41]:
!pip install sentencepiece

In [43]:
# Create a Gradio Interface with parallel inputs
iface = gr.Interface(
    fn=summary1,  # List of functions for parallel processing
    inputs=gr.Textbox(lines=20, label="Paste some text here"),
    outputs=["text"],  # Two text outputs for the results
    title="Compare 2 AI Summarizers"
)

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://38b6c94d716b0d9461.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 489, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-p

In [40]:
# Create a Gradio Interface with parallel inputs
iface = gr.Interface(
    fn=summary1,summary2  # List of functions for parallel processing
    inputs=gr.Textbox(lines=20, label="Paste some text here"),
    outputs=["text"],  # Two text outputs for the results
    title="Compare 2 AI Summarizers"
)

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://7a883f8feddaa34015.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 489, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\njoki\AppData\Local\Programs\Python\Python311\Lib\site-p